In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [33]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [34]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [35]:
df2 = df[df['koi_disposition'] != 'CANDIDATE']

In [36]:
df_real = df[df['koi_disposition'] == 'CANDIDATE']
real_feat = df_real[['koi_period','koi_time0bk','koi_impact','koi_depth','koi_prad','koi_teq','koi_insol','koi_steff','koi_duration','koi_slogg','koi_srad','ra','dec','koi_kepmag']]
real_feat.head()

,koi_period,koi_time0bk,koi_impact,koi_depth,koi_prad,koi_teq,koi_insol,koi_steff,koi_duration,koi_slogg,koi_srad,ra,dec,koi_kepmag
29,4.959319,172.258529,0.831,9802.0,12.21,1103,349.40,5712,2.22739,4.359,1.082,292.16705,48.727589,15.263
47,40.419504,173.564690,0.911,6256.0,7.51,467,11.29,5446,3.36200,4.507,0.781,294.31686,50.080231,15.487
50,7.240661,137.755450,1.198,556.4,19.45,734,68.63,5005,0.55800,4.595,0.765,293.83331,50.230350,15.334
51,3.435916,132.662400,0.624,23.2,0.55,1272,617.61,5779,3.13300,4.339,1.087,287.88733,46.276241,12.791
59,1.626630,169.820171,1.260,1620.6,32.43,1547,1355.88,5735,2.11450,4.410,0.973,294.36819,38.310280,15.279


# Select your features (columns)

In [37]:
# Set features. This will also be used as your x values.
selected_features = df2[['koi_period','koi_time0bk','koi_impact','koi_depth','koi_prad','koi_teq','koi_insol','koi_steff','koi_duration','koi_slogg','koi_srad','ra','dec','koi_kepmag']]
selected_features.head()

,koi_period,koi_time0bk,koi_impact,koi_depth,koi_prad,koi_teq,koi_insol,koi_steff,koi_duration,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,54.418383,162.513840,0.586,874.8,2.83,443,9.11,5455,4.50700,4.467,0.927,291.93423,48.141651,15.347
1,19.899140,175.850252,0.969,10829.0,14.60,638,39.30,5853,1.78220,4.544,0.868,297.00482,48.134129,15.436
2,1.736952,170.307565,1.276,8079.2,33.46,1395,891.96,5805,2.40641,4.564,0.791,285.53461,48.285210,15.597
3,2.525592,171.595550,0.701,603.3,2.75,1406,926.16,6031,1.65450,4.438,1.046,288.75488,48.226200,15.509
4,4.134435,172.979370,0.762,686.0,2.77,1160,427.65,6046,3.14020,4.486,0.972,296.28613,48.224670,15.714


In [38]:
y = df2['koi_disposition']

In [39]:
y2 = y.replace({'CONFIRMED': 1,"FALSE POSITIVE":0})

y3 = y2.astype(int).values.reshape(-1, 1)

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, y3, random_state=42)

In [41]:
X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_depth,koi_prad,koi_teq,koi_insol,koi_steff,koi_duration,koi_slogg,koi_srad,ra,dec,koi_kepmag
979,41.077297,379.128480,0.030,1216.8,3.25,523,17.74,5665,5.52500,4.381,0.938,289.07703,51.761822,14.907
6201,13.639480,144.531439,1.033,81103.0,44.00,680,50.51,5461,3.12422,4.517,0.846,292.21732,49.707901,15.852
5648,14.854416,136.559783,0.570,77410.0,29.77,798,95.83,6214,5.13753,4.444,1.022,282.42578,43.884331,14.558
229,10.681695,131.550820,0.646,307.8,2.30,897,152.75,5553,3.60500,4.221,1.250,292.58044,46.097389,15.065
1302,12.509458,136.992170,0.626,155.3,1.54,875,138.54,5949,3.47900,4.303,1.184,295.42200,38.735691,13.860


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [42]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(y_train)

In [43]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [44]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model0 = LinearRegression()
model0.fit(X_train_scaled, y_train_scaled)

predictions = model0.predict(X_test_scaled)
r2 = model0.score(X_test_scaled, y_test_scaled)
MSE = mean_squared_error(y_test_scaled, predictions)

print(f"MSE: {MSE}, R2: {r2}")

MSE: 0.17111323774304824, R2: 0.23759442709940845


In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

model1 = LogisticRegression()
model1.fit(X_train_scaled, y_train_scaled)

predictions = model1.predict(X_test_scaled)
r2 = model1.score(X_test_scaled, y_test_scaled)
MSE = mean_squared_error(y_test_scaled, predictions)

print(f"MSE: {MSE}, R2: {r2}")

MSE: 0.24660633484162897, R2: 0.753393665158371


/Users/gretapfundt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gretapfundt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [46]:
from sklearn.linear_model import Lasso

model2 = Lasso(alpha=.01)
model2.fit(X_train_scaled, y_train_scaled)
predictions = model2.predict(X_test_scaled)

r22 = model2.score(X_test_scaled, y_test_scaled)
MSE2 = mean_squared_error(y_test_scaled, predictions)

print(f"MSE: {MSE2}, R2: {r22}")

MSE: 0.21584571816868575, R2: 0.03828610463411053


In [47]:
from sklearn.linear_model import Ridge

### BEGIN SOLUTION
model3 = Ridge(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = model3.predict(X_test_scaled)

MSE3 = mean_squared_error(y_test_scaled, predictions)
r23 = model3.score(X_test_scaled, y_test_scaled)
### END SOLUTION

print(f"MSE: {MSE3}, R2: {r23}")

MSE: 0.17112323382617367, R2: 0.23754988893519322


In [48]:
from sklearn.linear_model import ElasticNet

### BEGIN SOLUTION
model4 = ElasticNet(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = model4.predict(X_test_scaled)

MSE4 = mean_squared_error(y_test_scaled, predictions)
r24 = model4.score(X_test_scaled, y_test_scaled)
### END SOLUTION

print(f"MSE: {MSE4}, R2: {r24}")

MSE: 0.2011837525096788, R2: 0.1036133959386557


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)